In [ ]:
# ÍNDICE :

# 1.EXPLORACIÓN DE IMÁGENES
# 2.PREPROCESADO DE IMÁGENES
# 3.MODELADO DE IMÁGENES

# 4.EXPLORACIÓN DE DATOS TABULARES
# 5.PREPROCESADO DE DATOS TABULARES
# 6.MODELADO DE DATOS TABULARES

# 7.EARLY FUSIÓN & LATE FUSIÓN

# **1 - EXPLORACIÓN DE IMÁGENES**

In [ ]:
import numpy as np
import pandas as pd

# Lectura CSV desde Google Drive
ruta_csv = '/content/drive/MyDrive/Colab Notebooks/airbnb-listings.csv'
airbnb_data = pd.read_csv(ruta_csv,  sep=';')

# Total de filas del dataframe
total_filas = len(airbnb_data)
print("El total de filas en el DataFrame es:", total_filas, "\n")

""" Queremos obtener información sobre los datos de imágenes y precios.
    En este caso, la exploración de NA lo realizo sobre todos los datos,
    puesto que es posible que existan NA en train y no en test o validacion """

# Verificar si existen valores nulos en la columna 'price'
if airbnb_data['Price'].isnull().any():
    print("La columna 'Price' contiene valores nulos.")
else:
    print("La columna 'Price' no contiene valores nulos.")

# Verificar si existen valores nulos en la columna 'price'
if airbnb_data['Thumbnail Url'].isnull().any():
    print("La columna 'Thumbnail Url' contiene valores nulos.")
else:
    print("La columna 'Thumbnail Url' no contiene valores nulos.")

print()

""" Cogemos solo con los datos de Madrid puesto que el resto de ciudades tiene muy pocas imágenes
    En el módulo de ML no era determinante la ubicación para el precio, pero creo que de esta forma
    elimino el poco ruido que pueda generar """

column_values = airbnb_data['City'].value_counts()
print(column_values.head(5))

# **2 - PREPROCESADO DE IMÁGENES**

In [ ]:
import numpy as np
import pandas as pd
import os

# Eliminamos NA de las columnas 'Thumbnail Url' y 'Price'
airbnb_data = airbnb_data.dropna(subset=['Thumbnail Url'])
airbnb_data = airbnb_data.dropna(subset=['Price'])
total_filas = len(airbnb_data)
print("El total de filas quitando los NA en 'Thumbnail Url' y 'Price' en el DataFrame es:", total_filas, "\n")

# Solo los datos de Madrid
airbnb_data = airbnb_data[airbnb_data['City'] == 'Madrid']
print(f'Dimensiones de solo Madrid: {airbnb_data.shape}', "\n")

""" Lo queremos es que el CSV contenga los mismos índices
    que el de las imágenes que se hayan podido descargar """

# Eliminamos todas las filas del CSV donde no se haya podido descargar la imagen
image_destination_dir = '/content/drive/MyDrive/imagenes_madrid'

# Lista para almacenar los índices de las filas válidas
valid_rows = []

# Itera sobre las filas del DataFrame
for index, row in airbnb_data.iterrows():
    # Obtiene el nombre de archivo basado en el índice
    image_filename = f'image_{index}.jpg'

    # Verifica si la imagen se descargó correctamente
    if os.path.isfile(os.path.join(image_destination_dir, image_filename)):
        valid_rows.append(index)

# Sobreescribimos el DataFrame solo con las filas válidas en imágenes y precios
airbnb_data = airbnb_data.loc[valid_rows]

ruta_imagenes = '/content/drive/MyDrive/imagenes_madrid/'
ruta_precios = '/content/drive/MyDrive/precios_madrid/'

# Obtener la lista de archivos en la carpeta
lista_imagenes = os.listdir(ruta_imagenes)
lista_precios = os.listdir(ruta_precios)

# Obtener el número de archivos en la carpeta
num_imagenes = len(lista_imagenes)
num_precios = len(lista_precios)

# Comprobamos que exista el mismo número de filas en CSV e imágenes
print(f"La carpeta de imagenes contiene {num_imagenes} imagenes.")
print(f"La carpeta de precios contiene {num_precios} precios.")
print("El dataframe contiene:", airbnb_data.shape[0], "filas")

In [ ]:
# Pasamos a un array el número de precios e imágenes que deseamos
import os
import cv2
import numpy as np

""" Descargamos las imágenes y precios de la carpeta de Google Drive y las pasamos a un array
    Posteriormente a arreglos de numpy para poder trabajar con ellos
    Añado una opción de descarga en la que puedes elegir el número que quieres utlizar
    de imágenes, ya que descargar todas lleva su tiempo """

image_folder = '/content/drive/MyDrive/imagenes_madrid'
price_folder = '/content/drive/MyDrive/precios_madrid'

image_files = os.listdir(image_folder)
price_files = os.listdir(price_folder)

images = []
prices = []

for i, (image_file, price_file) in enumerate(zip(image_files, price_files)):
    if i >= 2000: # Elegimos cantidad de imágenes
        break
    image_path = os.path.join(image_folder, image_file)
    price_path = os.path.join(price_folder, price_file)

    image = cv2.imread(image_path)
    images.append(image)

    price = np.loadtxt(price_path)
    prices.append(price)

# Convertir las listas a arreglos numpy
images = np.array(images)
prices = np.array(prices)

""" # Pasamos a un array precios e imágenes --> todas las imágenes
import os
import cv2
import numpy as np

image_folder = '/content/drive/MyDrive/imagenes_madrid'
price_folder = '/content/drive/MyDrive/precios_madrid'

image_files = os.listdir(image_folder)
price_files = os.listdir(price_folder)

images = []
prices = []
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    images.append(image)

for price_file in price_files:
    price_path = os.path.join(price_folder, price_file)
    price = np.loadtxt(price_path)
    prices.append(price)

# Convertir las listas a arreglos numpy
images = np.array(images)
prices = np.array(prices) """

In [ ]:
# Divisón, redimensión y normalización de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Semilla
SEED = 5

# Dividimos los datos en conjuntos de entrenamiento, validación y test
X_train, X_test, y_train, y_test = train_test_split(images, prices, test_size=0.2, random_state=SEED)
validation_split = 0.15
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, random_state=SEED)

""" Vamos usar redes convolucionales para las imágenes, probaremos una red desde cero y posteriormente
    un modelo preentrenado,tanto en forma Tranfer-learnig como Fine-tuning """

""" Dadas las limitaciones computacionales de mi ordenador y el gran número de imágenes a procesar
    quería la arquitectura SqueezeNet ya que es una red muy ligera y con buenos resultados,
    pero he tenido problemas con las versiones por lo que probaremos con el modelo ResNet que
    ocupa menos espacio y permite dimensiones más pequeñas que otros modelos """

# Pasamos a float
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Redimensión y normalización
input_shape = (180, 180, 3)

X_train_resized = []
for img in X_train:
  X_train_resized.append(np.resize(img, input_shape) / 255.)
X_train_resized = np.array(X_train_resized)
print("Forma de X_train en imágenes:", X_train_resized.shape)

X_val_resized = []
for img in X_val:
  X_val_resized.append(np.resize(img, input_shape) / 255.)
X_val_resized = np.array(X_val_resized)
print("Forma de X_val en imágenes:", X_val_resized.shape)

X_test_resized = []
for img in X_test:
  X_test_resized.append(np.resize(img, input_shape) / 255.)
X_test_resized = np.array(X_test_resized)
print("Forma de X_test en imágenes:", X_test_resized.shape)

# Conversión de etiquetas
scaler = MinMaxScaler()
y_train_resized = scaler.fit_transform(y_train.reshape(-1, 1))
y_val_resized = scaler.transform(y_val.reshape(-1, 1))
y_test_resized = scaler.transform(y_test.reshape(-1, 1))

# **3 - MODELADO DE IMÁGENES**

In [ ]:
# Búsqueda mejores Hiperparámetros
from hyperopt import fmin, tpe, hp, Trials
from tensorflow.keras import callbacks, optimizers, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
import tensorflow as tf

""" Además de estos hiperparámetros podemos buscar otros
    como units o layers , pero tardaría bastante más """

# Definir la función de entrenamiento y evaluación con los hiperparámetros como argumentos
def train_and_evaluate(params):

    # Construir el modelo con los hiperparámetros
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    last = base_model.layers[-1].output
    x = Flatten()(last)
    x = Dense(1000, activation='relu')(x)
  # x = Dropout(params['dropout_rate'])(x)
    x = Dense(1, activation='linear', name='regression')(x)
    model = Model(base_model.input, x)

    # Compilamos el modelo con Adam como optimizador
    model.compile(optimizer=Adam(params['learning_rate']), loss='mean_squared_error', metrics=['mae'])

    # Definir Early Stopping
    early_stopping =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, mode='auto')

    # Entrenanamos el modelo con los hiperparámetros
    history = model.fit(X_train_resized, y_train_resized,
                        validation_data=(X_val_resized, y_val_resized),
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        verbose=0,
                        callbacks=[early_stopping])

    # Obtener el valor de val_loss en la última época
    val_loss = history.history['val_loss'][-1]

    # Devolver el val_loss a minimizar
    return val_loss

# Definir el espacio de búsqueda de hiperparámetros
space = {
  # 'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5), -- > no tenemos overfitting
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.01),
    'epochs': hp.choice('epochs', [2, 4]),
    'batch_size': hp.choice('batch_size', [5, 10])
}

""" El space lo ajustamos en base a las pruebas que hemos
    ido realizando """

# Función objetivo para minimizar el val_loss directamente
def objective(params):
    val_loss = train_and_evaluate(params)
    return val_loss

# Configurar la búsqueda con Hyperopt
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials)

# Imprimir los mejores hiperparámetros encontrados
print('Mejores hiperparámetros:')
print(best)

In [ ]:
# Red Convolucional
from tensorflow.keras import optimizers, Model, layers
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
import tensorflow as tf

""" He decidido usar una regresión lineal para predecir el precio de las casas porque creo que
    el objetivo es predecir el precio de las habitaciones. Si lo clasifico en categorías
    estaríamos buscando tendencias de precios, pero no su valor real.
    Por otro lado, considero que teniendo 8433 filas no es necesario realizar 'Data augmentation' """

""" Con una red convolucional creada desde cero se consiguen peores resutlados
    que con modelos preentrenados, pero sobretodo consume bastante más RAM """

epochs = 3
batch_size = 2
lr = 0.001

# Inicializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(5, 5), activation='relu', input_shape=(input_shape)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Definimos una segunda capa convolucional
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear', name='regression'))

# Compilar el modelo
model.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mae'])

# Entrenamos el modelo
model.fit(X_train_resized, y_train_resized, validation_data=(X_val_resized, y_val_resized), epochs=epochs, batch_size=batch_size)

# Evaluamos el MSE y el MAE en test
scores = model.evaluate(X_test_resized, y_test_resized, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1])

# Predicciones en train
y_train_pred = model.predict(X_train_resized)
r2_train = r2_score(y_train_resized, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict(X_test_resized)
r2_test = r2_score(y_test_resized, y_test_pred)
print(f'R^2 Test: {r2_test}', "\n")

# Deshacer el escalado de etiquetas
""" y_train = scaler.inverse_transform(y_train_resized)
    y_val = scaler.inverse_transform(y_val_resized)
    y_test = scaler.inverse_transform(y_test_resized) """

In [ ]:
# Transfer learning
from tensorflow.keras import optimizers, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.applications import ResNet50

""" Dentro de Resnet exiten varias versiones en función de las capas (18-152),
    cogeremos algo intermedio en este caso Resnet50 """

# Hiperparámetros
epochs = 3
batch_size = 2
lr = 0.001

# Construimos el modelo base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Congelamos todas las capas de nuestro base_model para que no se entrenen
for layer in base_model.layers:
  layer.trainable = False

# Cogemos la última capa del model y le añadimos nuestra capa de regresión lineal
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu')(x)
# x = Dropout(0.2161074859485292)(x) ---> no tenemos overfitting
x = Dense(1, activation='linear', name='regression')(x)  # Capa de regresión lineal con 1 unidad
model = Model(base_model.input, x)

""" No estoy teniendo overfitting por lo que no implementaremos nigún método
    de regularización (Lasso/Ridge) ni Dropout """

# Compilamos el modelo
model.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mae'])  # Utilizamos MSE y MAE para regresión

# Entrenamos el modelo
model.fit(X_train_resized, y_train_resized, validation_data=(X_val_resized, y_val_resized), epochs=epochs, batch_size=batch_size)

# Evaluamos el MSE y el MAE en test
scores = model.evaluate(X_test_resized, y_test_resized, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1])

# Predicciones en train
y_train_pred = model.predict(X_train_resized)
r2_train = r2_score(y_train_resized, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict(X_test_resized)
r2_test = r2_score(y_test_resized, y_test_pred)
print(f'R^2 Test: {r2_test}', "\n")

# Deshacer el escalado de etiquetas
""" y_train = scaler.inverse_transform(y_train_resized)
    y_val = scaler.inverse_transform(y_val_resized)
    y_test = scaler.inverse_transform(y_test_resized) """

In [ ]:
# Fine-tuning
from tensorflow.keras import optimizers, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import mean_squared_error, r2_score

""" Mismos pasos que en Transfer-learning, pero en este caso le dejamos
    la última capa del modelo para que entrene """

""" Resultados bastante similares que con Transfer-learning """

""" Como conclusión diría que en redes convolucionales he conseguido mejores
    resultados con menos imágenes y más shape que con todas las imágenes y
    menos resolución """

# Hiper-parámetros
epochs = 3
batch_size = 2
lr = 0.001

# Construimos el modelo base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# permitimos que, además de la etapa de clasificación, se entrenen también el último bloque convolucional
for layer in base_model.layers:
  layer.trainable = False
  if layer.name == "conv5_block3_3_conv":
    layer.trainable = True

# cogemos la última capa del model y le añadimos nuestra capa de regresión lineal
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(500, activation='relu')(x)
x = Dense(1, activation='linear', name='regression')(x)  # Capa de regresión lineal con 1 unidad
# x = Dropout(0.2161074859485292)(x) ---> no tenemos overfitting
model = Model(base_model.input, x)

# Compilamos el modelo
model.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mae'])  # Utilizamos MSE y MAE para regresión

# Entrenamos el modelo
model.fit(X_train_resized, y_train_resized, validation_data=(X_val_resized, y_val_resized),  epochs=epochs, batch_size=batch_size)

# Evaluamos el MSE y el MAE en test
scores = model.evaluate(X_test_resized, y_test_resized, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1])

# Predicciones en train
y_train_pred = model.predict(X_train_resized)
r2_train = r2_score(y_train_resized, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict(X_test_resized)
r2_test = r2_score(y_test_resized, y_test_pred)
print(f'R^2 Test: {r2_test}')

# Deshacer el escalado de etiquetas
""" y_train = scaler.inverse_transform(y_train_resized)
    y_val = scaler.inverse_transform(y_val_resized)
    y_test = scaler.inverse_transform(y_test_resized) """

# **4 - EXPLORACIÓN DE DATOS TABULARES**

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Selección de columnas propuestas en la práctica
selected_columns = ['Price', 'Property Type', 'Room Type', 'Cancellation Policy', 'Accommodates', 'Bathrooms',
                    'Bedrooms', 'Beds', 'Guests Included', 'Extra People', 'Minimum Nights',
                    'Maximum Nights', 'Number of Reviews', 'Host Total Listings Count']

# Cogemos el mismo DataFrame ya procesado en imágenes con las columnas seleccionadas
airbnb_data = airbnb_data[selected_columns]

""" Primero arrancar la celda de preprocesado de imágenes para tener las mismas filas
    Podemos elegir menos filas, pero tienen que ser las mismas que imágenes,
    de lo contrario early fusión y late fusión no funcionarán """

# Elegimos el numero de filas que deseamos
# Tiene que coincidir con el número de imágenes
airbnb_data = airbnb_data.head(2000)

# Semilla
SEED = 40

# Dividimos los datos en conjuntos de entrenamiento, validación y test
train_tab, test_tab = train_test_split(airbnb_data, test_size=0.2, random_state=SEED)
train_tab, val_tab = train_test_split(train_tab, test_size=0.15, random_state=SEED)

# Verificamos formas de los conjuntos de entrenamiento, validación y test
print("Forma de train en datos tabulares:", train_tab.shape)
print("Forma de val en datos tabulares:", val_tab.shape)
print("Forma de test en datos tabulares:", test_tab.shape, "\n")

airbnb_data.head(5)

In [ ]:
# Mapa de correlación entre variables de train
import matplotlib.pyplot as plt
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(train_tab.drop(['Price', 'Room Type', 'Property Type', 'Cancellation Policy'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

""" Variables algo correlacionadas'. Si tuvieramos overfitting
    utilizaríamos métodos de regularización """

# Información de train
train_tab.info()

print()

""" Vemos que no todos los datos son del mismo tipo
    Les aplicaremos a todos tipo 'float' """

# Porcentaje de nulos de train
pd.set_option('display.max_rows', None)

valores_nulos = train_tab.isnull().sum()
porcentaje_nulos = (valores_nulos / total_filas) * 100
print("Porcentaje de nulos:", porcentaje_nulos, "\n")

""" Al existir pocos porcentajes de nulos los sustituiremos por la moda """

# **5 - PREPROCESADO DATOS TABULARES**

In [ ]:
# Tomamos decisiones en base a lo que hemos visto en train
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Preprocesamiento de test

# Categorizamos variables
label_encoder = LabelEncoder()
train_tab['Room Type'] = label_encoder.fit_transform(train_tab['Room Type'])
train_tab['Property Type'] = label_encoder.fit_transform(train_tab['Property Type'])
train_tab['Cancellation Policy'] = label_encoder.fit_transform(train_tab['Cancellation Policy'])

# Convertir las columnas de tipo object a int64
float_columns = train_tab.select_dtypes(include=['object']).columns
train_tab[float_columns] = train_tab[float_columns].astype('float64')

# Rellenamos valores faltantes con la moda
column_modes = train_tab.mode().iloc[0]
train_tab = train_tab.fillna(column_modes)

# Obtener las variables de entrada y salida
X_train_tab = train_tab.iloc[:, 1:].values  # Variables de entrada
y_train_tab = train_tab.iloc[:, 0].values  # Variable de salida 'Price'

# Normalización de variables de entrada y etiquetas
scaler_entradas = MinMaxScaler()
X_train_tab = scaler_entradas.fit_transform(X_train_tab)

scaler_etiquetas = MinMaxScaler()
y_train_tab = scaler_etiquetas.fit_transform(y_train_tab.reshape(-1, 1))

# Preprocesamiento de val

# Categorizamos variables
val_tab['Room Type'] = label_encoder.fit_transform(val_tab['Room Type'])
val_tab['Property Type'] = label_encoder.fit_transform(val_tab['Property Type'])
val_tab['Cancellation Policy'] = label_encoder.fit_transform(val_tab['Cancellation Policy'])

# Convertir las columnas de tipo object a int64
float_columns = val_tab.select_dtypes(include=['object']).columns
val_tab[float_columns] = val_tab[float_columns].astype('float64')

# Rellenamos valores faltantes con la moda
column_modes = val_tab.mode().iloc[0]
val_tab = val_tab.fillna(column_modes)

# Obtener las variables de entrada y salida
X_val_tab = val_tab.iloc[:, 1:].values  # Variables de entrada
y_val_tab = val_tab.iloc[:, 0].values # Variable de salida 'Price'

# Normalización de variables de entrada y etiquetas
X_val_tab = scaler_entradas.transform(X_val_tab)
y_val_tab = scaler_etiquetas.transform(y_val_tab.reshape(-1, 1))

# Preprocesamiento de test

# Categorizamos variables
test_tab['Room Type'] = label_encoder.fit_transform(test_tab['Room Type'])
test_tab['Property Type'] = label_encoder.fit_transform(test_tab['Property Type'])
test_tab['Cancellation Policy'] = label_encoder.fit_transform(test_tab['Cancellation Policy'])

# Convertir las columnas de tipo object a int64
float_columns = test_tab.select_dtypes(include=['object']).columns
test_tab[float_columns] = test_tab[float_columns].astype('float64')

# Rellenamos valores faltantes con la moda
column_modes = test_tab.mode().iloc[0]
test_tab = test_tab.fillna(column_modes)

# Obtener las variables de entrada y salida
X_test_tab = test_tab.iloc[:, 1:].values  # Variables de entrada
y_test_tab = test_tab.iloc[:, 0].values  # Variable de salida 'Price'

# Normalización de variables de entrada y etiquetas
X_test_tab = scaler_entradas.transform(X_test_tab)
y_test_tab = scaler_etiquetas.transform(y_test_tab.reshape(-1, 1))

# **6 - MODELADO DATOS TABULARES**

In [ ]:
# Búsqueda mejores Hiperparámetros
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, Trials

# Definir la función de entrenamiento y evaluación con los hiperparámetros como argumentos
def train_and_evaluate(params):

    # Creamos la red neuronal
    model = Sequential()
    model.add(Flatten())

    # Agregamos las capas ocultas
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1, input_shape=(13,), activation='linear', name='regression'))

    # Compilamos el modelo con Adam como optimizador
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mean_squared_error', metrics=['mae'])

    # Definimos Early Stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, mode='auto')

    # Entrenamos el modelo
    history = model.fit(X_train_tab, y_train_tab,
                        validation_data=(X_val_tab, y_val_tab),
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        verbose=0,
                        callbacks=[early_stopping])

    # Obtenemos el val_loss en la última época
    val_loss = history.history['val_loss'][-1]

    # Devolvemos el val_loss a minimizar
    return val_loss

# Definir el espacio de búsqueda de hiperparámetros
space = {
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.01),
    'epochs': hp.choice('epochs', [3, 5]),
    'batch_size': hp.choice('batch_size', [2, 5])
}

# Función objetivo para minimizar el val_loss
def objective(params):
    val_loss = train_and_evaluate(params)
    return val_loss

# Configurar la búsqueda con Hyperopt
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials)

# Imprimir los mejores hiperparámetros encontrados
print('Mejores hiperparámetros:')
print(best)

In [ ]:
# Creamos la red neuronal
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential

""" Tal y como hemos procedido en las imágenes usaremos regresión lineal
    para las predicciones """

# Hiper-parámetros
n_epochs = 3
batch_size = 2
lr = 0.001

model = Sequential()
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1, input_shape=(13,), activation='linear', name='regression'))

# Compilamos el modelo
model.compile(optimizer=Adam(lr), loss="mean_squared_error", metrics=["mae"])

# Entrenamos el modelo
model.fit(X_train_tab, y_train_tab, validation_data=(X_val_tab, y_val_tab), epochs=n_epochs, batch_size=batch_size)

# Evaluamos el MSE y el MAE en el conjunto de prueba
scores = model.evaluate(X_test_tab, y_test_tab, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1], "\n")

# Predicciones en train
y_train_pred = model.predict(X_train_tab)
r2_train = r2_score(y_train_tab, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict(X_test_tab)
r2_test = r2_score(y_test_tab, y_test_pred)
print(f'R^2 Test: {r2_test}')

# Deshacer el escalado de etiquetas
""" y_train_tab = scaler.inverse_transform(y_train_tab)
    y_val_tab = scaler.inverse_transform(y_val_tab)
    y_test_tab = scaler.inverse_transform(y_test_tab) """

# **7 - EARLY FUSION  &  LATE FUSIÓN**

---



In [ ]:
# Búsqueda mejores Hiperparámetros
import numpy as np
from tensorflow.keras import callbacks, optimizers, Model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.applications import ResNet50
from hyperopt import fmin, tpe, hp, Trials

# Definir la función de entrenamiento y evaluación con los hiperparámetros como argumentos
def train_and_evaluate(params):

    # Construir el modelo base de imágenes
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Congelar todas las capas del modelo base para que no se entrenen
    for layer in base_model.layers:
        layer.trainable = False

    n_features = 13

    # Capa de entrada para características tabulares
    input_tabular = Input(shape=(n_features,))

    # Capa de aplanado y capas densas para características de imágenes
    image_features = base_model.output
    image_features = Flatten()(image_features)
    image_features = Dense(100)(image_features)

    # Capa de fusión de características
    merged_features = Concatenate()([input_tabular, image_features])

    # Capa de regresión lineal
    output_layer = Dense(1, activation='linear', name='regression')(merged_features)

    # Modelo final de early fusion
    model = Model(inputs=[base_model.input, input_tabular], outputs=output_layer)

    # Compilar el modelo
    model.compile(optimizer=Adam(params['learning_rate']), loss='mean_squared_error', metrics=['mae'])

    # Definir Early Stopping
    early_stopping =  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, mode='auto')

    # Entrenar el modelo
    model.fit([X_train_resized, X_train_tab], y_train_tab,
              validation_data=([X_val_resized, X_val_tab], y_val_tab),
              epochs = params['epochs'], batch_size = params['batch_size'],
              verbose=0, callbacks=[early_stopping])

    # Evaluar el modelo en el conjunto de validación
    val_loss = model.evaluate([X_val_resized, X_val_tab], y_val_tab, verbose=0)[0]

    # Devolver el val_loss a minimizar
    return val_loss

# Definir el espacio de búsqueda de hiperparámetros
space = {
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.01),
    'epochs': hp.choice('epochs', [3, 5]),
    'batch_size': hp.choice('batch_size', [2, 5])
}

# Función objetivo para minimizar el val_loss
def objective(params):
    val_loss = train_and_evaluate(params)
    return val_loss

# Configurar la búsqueda con Hyperopt
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials)

# Imprimir los mejores hiperparámetros encontrados
print('Mejores hiperparámetros:')
print(best)

In [ ]:
# Early-fusión
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import mean_squared_error, r2_score

""" En este caso, realizamos la extracción de características de imágenes y de
    datos tabulares para unirlas y realizar una predicción con regresión lineal """

""" También utlizaremos Resnet50 como modelo preentrenado """

""" Resultados bastante peores que con Late-fusión """

# Hiper-parámetros
epochs = 3
batch_size = 2
lr = 0.001

# Construimos el modelo base de imágenes
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Congelamos todas las capas del modelo base para que no se entrenen
for layer in base_model.layers:
    layer.trainable = False

n_features = 13

# Capa de entrada para características tabulares
input_tabular = Input(shape=(n_features,))

# Capa de aplanado y capas densas para características de imágenes
image_features = base_model.output
image_features = Flatten()(image_features)

# Capa de fusión de características
merged_features = Concatenate()([input_tabular, image_features])

# Capa de regresión lineal
output_layer = Dense(1, activation='linear', name='regression')(merged_features)

# Modelo final de early fusion
model = Model(inputs=[base_model.input, input_tabular], outputs=output_layer)

# Compilamos el modelo
model.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mae'])

# Entrenamos el modelo
model.fit([X_train_resized, X_train_tab], y_train_tab, validation_data=([X_val_resized, X_val_tab], y_val_tab), epochs=epochs, batch_size=batch_size)

# Evaluamos el modelo en el conjunto de prueba
scores = model.evaluate([X_test_resized, X_test_tab], y_test_tab, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1])

# Predicciones en train
y_train_pred = model.predict([X_train_resized, X_train_tab])
r2_train = r2_score(y_train_tab, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict([X_test_resized, X_test_tab])
r2_test = r2_score(y_test_tab, y_test_pred)
print(f'R^2 Test: {r2_test}')

# Deshacer el escalado de etiquetas
""" y_train_tab = scaler.inverse_transform(y_train_tab)
    y_val_tab = scaler.inverse_transform(y_val_tab)
    y_test_tab = scaler.inverse_transform(y_test_tab) """

In [ ]:
# Late-fusion
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Concatenate, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import mean_squared_error, r2_score

""" Parecido al caso de Eaely-fusión, pero ahora no unimos las características,
    sino que unimos las predicciones que han realizado ambos por separado en una
    nueva predicción """

""" Es curioso que al probar como etiqueta a predecir las imágenes en lugar de datos
    tabulares tampoco han sido unos resultados muy por debajo, aun teniendo mucha
    menos información para aprender. No obstante, he preferido dejar los datos
    tabulares como etiqueta de predicción """

# Hiper-parámetros
epochs = 3
batch_size = 2
lr = 0.001

# Construir el modelo base de imágenes
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Congelar todas las capas del modelo base para que no se entrenen
for layer in base_model.layers:
  layer.trainable = False

n_features = 13

# Capa de entrada para características tabulares
input_tabular = Input(shape=(n_features,))

# Capas densas para características tabulares
tabular_features = Dense(64, activation='relu')(input_tabular)

# Modelo para características tabulares
tabular_model = Model(inputs=input_tabular, outputs=tabular_features)

# Capas densas para características de imágenes
image_features = base_model.output
image_features = Flatten()(image_features)
image_features = Dense(1000, activation='relu')(image_features)
# image_features = Dropout(0.23204909844501467)(image_features)

# Modelo para características de imágenes
image_model = Model(inputs=base_model.input, outputs=image_features)

# Concatenar las salidas de los modelos
merged_features = Concatenate()([image_model.output, tabular_model.output])

# Capa de regresión lineal
output_layer = Dense(1, activation='linear', name='regression')(merged_features)

# Modelo final de late fusion
model = Model(inputs=[image_model.input, tabular_model.input], outputs=output_layer)

# Compilar el modelo
model.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mae'])

# Entrenar el modelo
model.fit([X_train_resized, X_train_tab], y_train_tab, validation_data=([X_val_resized, X_val_tab], y_val_tab), epochs=epochs, batch_size=batch_size)

# Evaluamos el modelo en el conjunto de prueba
scores = model.evaluate([X_test_resized, X_test_tab], y_test_tab, verbose=1)
print('Test MSE:', scores[0])
print('Test MAE:', scores[1])

# Predicciones en train
y_train_pred = model.predict([X_train_resized, X_train_tab])
r2_train = r2_score(y_train_tab, y_train_pred)
print(f'R^2 Train: {r2_train}')

# Predicciones en test
y_test_pred = model.predict([X_test_resized, X_test_tab])
r2_test = r2_score(y_test_tab, y_test_pred)
print(f'R^2 Test: {r2_test}')

# Deshacer el escalado de etiquetas
""" y_train_tab = scaler.inverse_transform(y_train_tab)
    y_val_tab = scaler.inverse_transform(y_val_tab)
    y_test_tab = scaler.inverse_transform(y_test_tab) """

In [ ]:
# Conclusión Final

""" En líneas generales creo que he estado condicionado por las limitaciones de la RAM a la hora de poder incluir más imágenes
    o aumentar su resolución para conseguir mejores resultados. Lo mismo con la búsqueda de hiperparámetros, pero entiendo que esto
    también será un problema del día a día en el trabajo de las empresas, asíque habrá que buscar ese equilibrio entre
    el tamaño de los datos y la capacidad del procesamiento para conseguir los mejores resultados """

In [ ]:
# Script de subida de imágenes y precios a Google Drive

""" import cv2
import os
from urllib import request, error

# Directorio de destino en Google Drive para las imágenes
image_destination_dir = '/content/drive/MyDrive/imagenes_madrid/'

# Directorio de destino en Google Drive para los datos de precios
price_destination_dir = '/content/drive/MyDrive/precios_madrid/'

# Crea los directorios de destino si no existen
if not os.path.exists(image_destination_dir):
    os.makedirs(image_destination_dir)

if not os.path.exists(price_destination_dir):
    os.makedirs(price_destination_dir)

# Itera sobre las filas del DataFrame
for index, row in airbnb_data.iterrows():
    # Obtiene la URL de la columna "Picture Url"
    url = row['Thumbnail Url']

    # Genera el nombre de archivo basado en el índice
    image_filename = f'image_{index}.jpg'

    # Descarga la imagen y guárdala en Google Drive
    try:
        request.urlretrieve(url, os.path.join(image_destination_dir, image_filename))
        print("Imagen descargada:", image_filename)

        # Obtiene el precio de la columna "Price"
        price = row['Price']

        # Genera el nombre de archivo para el precio basado en el índice
        price_filename = f'price_{index}.txt'

        # Guarda el precio en un archivo de texto en Google Drive
        try:
            with open(os.path.join(price_destination_dir, price_filename), 'w') as f:
                f.write(str(price))
            print("Precio guardado:", price_filename)
        except Exception as e:
            print("Error al guardar el precio", price_filename, ":", str(e))

    except error.URLError as e:
        print("Error al descargar la imagen", image_filename, ":", str(e))
    except Exception as e:
        print("Error inesperado al descargar la imagen", image_filename, ":", str(e))
  """